In [5]:
import numpy as np
import pandas as pd
import glob
import os

# 获取所有 CSV 文件的路径
datapath = "/home/zihan/dataset/zhaokedata/200113plc1p2"
csv_files = glob.glob(os.path.join(datapath, "*.csv"))

# 初始化一个字典用于存储所有数据
data = {}

# 读取所有 CSV 文件，并将时间信息添加到字典中
for file in csv_files:
    # 从文件名中提取时间信息
    filename = os.path.basename(file)
    time_point = int(filename.split('_')[1])
    
    # 读取 CSV 文件
    df = pd.read_csv(file)
    
    # 添加数据到字典中
    for index, row in df.iterrows():
        label = row['nucleus_label']
        name = row['nucleus_name']
        if (label, name) not in data:
            data[(label, name)] = {}
        data[(label, name)][time_point] = row[['x_256', 'y_356', 'z_214', 'volume', 'surface']].values

# 获取所有的细胞和时间点
cells = list(data.keys())
time_points = sorted({time for times in data.values() for time in times})

# 初始化张量，维度为 (num_cells, num_time_points, num_features)
num_cells = len(cells)
num_time_points = len(time_points)
num_features = 5  # x, y, z, volume, surface
tensor = np.full((num_cells, num_time_points, num_features), np.nan)

# 填充张量
for i, cell in enumerate(cells):
    for j, time in enumerate(time_points):
        if time in data[cell]:
            tensor[i, j, :] = data[cell][time]

# 打印张量形状
print(f"Tensor shape: {tensor.shape}")

# 示例输出部分张量数据
print(tensor[:5, :5, :])


Tensor shape: (1288, 255, 5)
[[[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]

 [[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]

 [[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]

 [[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]

 [[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]]


In [6]:
import numpy as np

def find_subtensors(tensor):
    depth, rows, cols = tensor.shape
    visited = np.zeros_like(tensor, dtype=bool)
    subtensors = []

    def find_max_subtensor(d, r, c):
        max_d, max_r, max_c = d, r, c
        while max_d < depth and not np.isnan(tensor[max_d, r, c]) and not visited[max_d, r, c]:
            max_r_temp = r
            while max_r_temp < rows and not np.isnan(tensor[max_d, max_r_temp, c]) and not visited[max_d, max_r_temp, c]:
                max_c_temp = c
                while max_c_temp < cols and not np.isnan(tensor[max_d, max_r_temp, max_c_temp]) and not visited[max_d, max_r_temp, max_c_temp]:
                    max_c_temp += 1
                max_r_temp += 1
                max_c = max(max_c, max_c_temp)
            max_d += 1
            max_r = max(max_r, max_r_temp)
        return (d, r, c, max_d - 1, max_r - 1, max_c - 1)

    for d in range(depth):
        for i in range(rows):
            for j in range(cols):
                if not np.isnan(tensor[d, i, j]) and not visited[d, i, j]:
                    d1, r1, c1, d2, r2, c2 = find_max_subtensor(d, i, j)
                    subtensors.append(((d1, r1, c1), (d2, r2, c2)))
                    for d_ in range(d1, d2 + 1):
                        for r in range(r1, r2 + 1):
                            for c in range(c1, c2 + 1):
                                visited[d_, r, c] = True

    return subtensors

# 示例三维张量
# tensor = np.array([
#     [
#         [1, 2, 3, np.nan, 5],
#         [6, np.nan, 8, 9, 10],
#         [np.nan, np.nan, 11, 12, 13],
#         [14, 15, 16, np.nan, np.nan]
#     ],
#     [
#         [17, 18, np.nan, 20, 21],
#         [np.nan, 23, 24, 25, 26],
#         [27, 28, 29, 30, np.nan],
#         [32, 33, 34, 35, 36]
#     ]
# ])

subtensors = find_subtensors(tensor)
for subtensor in subtensors:
    print(f"Subtensor from {subtensor[0]} to {subtensor[1]}")


Subtensor from (0, 34, 0) to (9, 52, 4)
Subtensor from (2, 53, 3) to (3, 53, 4)
Subtensor from (6, 53, 3) to (7, 53, 4)
Subtensor from (10, 35, 0) to (23, 83, 4)
Subtensor from (12, 34, 0) to (13, 34, 2)
Subtensor from (16, 27, 0) to (17, 34, 4)
Subtensor from (18, 28, 0) to (19, 34, 4)
Subtensor from (20, 30, 0) to (21, 34, 4)
Subtensor from (23, 84, 3) to (23, 85, 4)
Subtensor from (24, 133, 0) to (31, 190, 4)
Subtensor from (24, 191, 3) to (25, 191, 4)
Subtensor from (25, 227, 3) to (25, 227, 4)
Subtensor from (28, 127, 0) to (31, 132, 4)
Subtensor from (32, 134, 0) to (60, 195, 4)
Subtensor from (34, 133, 0) to (47, 133, 4)
Subtensor from (36, 130, 0) to (43, 132, 4)
Subtensor from (38, 128, 0) to (39, 129, 4)
Subtensor from (40, 196, 3) to (40, 196, 4)
Subtensor from (42, 129, 0) to (43, 129, 2)
Subtensor from (44, 131, 0) to (47, 132, 4)
Subtensor from (46, 126, 0) to (47, 130, 4)
Subtensor from (50, 129, 0) to (55, 133, 4)
Subtensor from (52, 125, 0) to (55, 128, 4)
Subtensor fr